In [8]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv


--2025-11-04 21:13:37--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: 'car_fuel_efficiency.csv.2'

     0K .......... .......... .......... .......... ..........  5% 59.4M 0s
    50K .......... .......... .......... .......... .......... 11% 51.1M 0s
   100K .......... .......... .......... .......... .......... 17% 56.6M 0s
   150K .......... .......... .......... .......... .......... 23% 63.1M 0s
   200K .......... .......... .......... .......... .......... 29% 68.4M 0s
   250K .......... .......... .......... .......... .......... 35%  232M 0s
   300K .......... .......... .......... .......... .......... 40%  258M 0s
   350K

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [10]:
df = pd.read_csv('car_fuel_efficiency.csv')


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB


In [17]:
categorical_columns = ["origin","fuel_type","drivetrain"]
numerical_columns = ["engine_displacement","num_cylinders","horsepower","vehicle_weight","acceleration","model_year","num_doors","fuel_efficiency_mpg"]

In [18]:
for i in numerical_columns:
    print(df[i].isnull().sum())
df[numerical_columns] = df[numerical_columns].fillna(0.0)


0
482
708
0
930
0
502
0


In [32]:
target = 'fuel_efficiency_mpg'  
y = df[target]                   
X = df.drop(columns=[target])  
X = pd.get_dummies(X, drop_first=True)


In [33]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=1
)

# Then split the temp set equally into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=1
)

In [34]:
regressor = DecisionTreeRegressor(max_depth=1, random_state=42)
regressor.fit(X, y)

DecisionTreeRegressor(max_depth=1, random_state=42)

In [36]:
split_feature_index = regressor.tree_.feature[0]
split_feature_name = X.columns[split_feature_index]
print("Feature used for splitting:", split_feature_name)

Feature used for splitting: vehicle_weight


In [40]:
# random forest
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict on validation set
y_pred = rf.predict(X_val)

# Compute RMSE
rmse = mean_squared_error(y_val, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 0.4604844432870149


C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [43]:
rmse_list = []

for n in range(10, 201, 10):  # 10 to 200 with step 10
    model = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    rmse_list.append((n, rmse))

# Print results
for n, rmse in rmse_list:
    print(f"n_estimators={n:3d}, RMSE={rmse:.3f}")

C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 an

n_estimators= 10, RMSE=0.460
n_estimators= 20, RMSE=0.446
n_estimators= 30, RMSE=0.440
n_estimators= 40, RMSE=0.439
n_estimators= 50, RMSE=0.437
n_estimators= 60, RMSE=0.436
n_estimators= 70, RMSE=0.436
n_estimators= 80, RMSE=0.436
n_estimators= 90, RMSE=0.436
n_estimators=100, RMSE=0.436
n_estimators=110, RMSE=0.435
n_estimators=120, RMSE=0.436
n_estimators=130, RMSE=0.435
n_estimators=140, RMSE=0.435
n_estimators=150, RMSE=0.436
n_estimators=160, RMSE=0.436
n_estimators=170, RMSE=0.436
n_estimators=180, RMSE=0.435
n_estimators=190, RMSE=0.435
n_estimators=200, RMSE=0.435


C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [45]:
depth_values = [10, 15, 20, 25]
results = {}

for depth in depth_values:
    rmses = []
    for n in range(10, 201, 10):
        model = RandomForestRegressor(
            n_estimators=n,
            max_depth=depth,
            random_state=1,
            n_jobs=-1
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        rmses.append(rmse)
    results[depth] = np.mean(rmses)
    print(f"max_depth={depth}, mean RMSE={np.mean(rmses):.3f}")

best_depth = min(results, key=results.get)
print("Best max_depth:", best_depth)

C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 an

max_depth=10, mean RMSE=0.436


C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 an

max_depth=15, mean RMSE=0.438


C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 an

max_depth=20, mean RMSE=0.438


C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 an

max_depth=25, mean RMSE=0.438
Best max_depth: 10


C:\Users\fulbe\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [46]:
# Train the model
rf = RandomForestRegressor(
    n_estimators=10,
    max_depth=20,
    random_state=1,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# Get feature importances
importances = pd.Series(rf.feature_importances_, index=X_train.columns)
importances.sort_values(ascending=False, inplace=True)
print(importances)

vehicle_weight                  0.959935
horsepower                      0.016012
acceleration                    0.011526
engine_displacement             0.003235
model_year                      0.003189
num_cylinders                   0.002366
num_doors                       0.001613
drivetrain_Front-wheel drive    0.000556
fuel_type_Gasoline              0.000555
origin_Europe                   0.000511
origin_USA                      0.000503
dtype: float64


In [53]:
import xgboost as xgb
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_val, label=y_val)

watchlist = [(train_data, 'train'), (valid_data, 'valid')]

# ------------------------------
# Model with eta = 0.3
# ------------------------------
params_1 = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1
}

print("Training model with eta = 0.3 ...")
model1 = xgb.train(params_1, train_data, num_boost_round=100, evals=watchlist, verbose_eval=False)

# Make predictions
preds1 = model1.predict(valid_data)

# Calculate RMSE
rmse1 = mean_squared_error(y_val, preds1, squared=False)
print("RMSE for eta=0.3:", rmse1)


# ------------------------------
# Model with eta = 0.1
# ------------------------------
params_2 = params_1.copy()
params_2['eta'] = 0.1

print("\nTraining model with eta = 0.1 ...")
model2 = xgb.train(params_2, train_data, num_boost_round=100, evals=watchlist, verbose_eval=False)

# Make predictions
preds2 = model2.predict(valid_data)

# Calculate RMSE
rmse2 = mean_squared_error(y_val, preds2, squared=False)
print("RMSE for eta=0.1:", rmse2)


# ------------------------------
# Compare results
# ------------------------------
if rmse1 < rmse2:
    print("\neta = 0.3 gives better RMSE")
elif rmse1 > rmse2:
    print("\neta = 0.1 gives better RMSE")
else:
    print("\nBoth give equal RMSE")

ModuleNotFoundError: No module named 'xgboost'